In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

while 'notebooks' in os.getcwd():
    os.chdir('..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from tqdm import tqdm
import torch

from time import time

from src.simulations import PathSimulator
from src.gpu.simulation_based_tree import SimulationBasedModel

# Simulation based trees 

### Black-Scholes approach

In [4]:
simulator = PathSimulator()

In [ ]:
kappa = 5.0
K = 10
r = 0.0
eta = 0.9
theta = 0.04
rho = 0.1
T = 0.25
v0 = theta
S0 = 10
N = 200
M = 10_000
dt = T/N

In [19]:
def black_scholes(S, K, T, r, sigma, option_type='call'):
    """
    Compute the Black-Scholes option price for a European call or put.
    
    Parameters:
    S (float): Current stock price
    K (float): Strike price
    T (float): Time to maturity (in years)
    r (float): Risk-free interest rate (annual)
    sigma (float): Volatility of the underlying asset (annual)
    option_type (str): 'call' for a call option, 'put' for a put option
    
    Returns:
    float: Black-Scholes option price
    """
    # Calculate d1 and d2
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        # Call option price
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        # Put option price
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be either 'call' or 'put'")
    
    return price

In [20]:
pricer = SimulationBasedModel(N, K, r, T/N, option_type='european')

In [ ]:
for n in [100, 200, 500,1000]:
    m = 1000*n

    print(f"\n-------------------- N = {n} -------------------------\n")

    for S0 in [8,9,10,11,12]:

        dt = T/n

        t0 = time()
        St = simulator.bs_gpu(r, T, m, n, np.sqrt(theta), S0, )
        vt = torch.ones_like(St) * theta
        vt = vt.to('cuda')

        pricer = SimulationBasedModel(n, K, r, T/n, option_type='european')

        option_price_proba = pricer.compute_option_prices_probabilities(
            St,
            vt,
            lambda x,k: torch.maximum(k-x,torch.tensor(0, device='cuda')),
            n,
            T,
            K,
            r,
            v0
        )
        option_price = pricer.compute_option_prices_counting(
            St, 
            lambda S, K: torch.maximum(K - S, torch.tensor(0.0, device="cuda"))
        )

        reference_values = black_scholes(S0, K, T, r, np.sqrt(theta), 'put')
        

        t1 = time()

        print(f"S0 = {S0}. Reference value: {reference_values}. Counting approach: {option_price.item()} . Probabilities approach: {option_price_proba}. ({round(t1-t0, 2)} s) ")



-------------------- N = 100 -------------------------

S0 = 8. Reference value: 2.0039914343421836. Counting approach: 1.9851127862930298 . Probabilities approach: 2.050159215927124. (0.24 s) 
S0 = 9. Reference value: 1.0712380896073679. Counting approach: 1.0536370277404785 . Probabilities approach: 1.0897748470306396. (0.22 s) 
S0 = 10. Reference value: 0.3987761167674506. Counting approach: 0.38805705308914185 . Probabilities approach: 0.3796975612640381. (0.22 s) 
S0 = 11. Reference value: 0.09539473918572217. Counting approach: 0.08631381392478943 . Probabilities approach: 0.07101371139287949. (0.22 s) 
S0 = 12. Reference value: 0.01473322632569618. Counting approach: 0.009321510791778564 . Probabilities approach: 0.004636270459741354. (0.22 s) 

-------------------- N = 200 -------------------------

S0 = 8. Reference value: 2.0039914343421836. Counting approach: 1.9951183795928955 . Probabilities approach: 2.0357561111450195. (0.5 s) 
S0 = 9. Reference value: 1.071238089607367

: 

In [ ]:
pricer.memo[0]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [20]:
transition_matrix = torch.zeros((bins_current.unique()[-1]+1, bins_next.unique()[-1] +1 ), dtype=torch.int64, device='cuda')

    # Use advanced indexing to accumulate counts
indices = torch.stack([bins_current, bins_next], dim=0)
transition_matrix.index_put_((bins_current, bins_next), torch.ones_like(bins_current), accumulate=True)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
transition_matrix/transition_matrix.sum(dim=1, keepdim=True)

tensor([[0.8930, 0.1070, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0970, 0.7061, 0.1869,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1677, 0.5646,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.5303, 0.1838, 0.0010],
        [0.0000, 0.0000, 0.0000,  ..., 0.2101, 0.6869, 0.0939],
        [0.0000, 0.0000, 0.0000,  ..., 0.0010, 0.1039, 0.8951]],
       device='cuda:0')

In [65]:
bins_current.unique()

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [67]:
bins_next.unique()

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [ ]:
100bi*101

10100

In [30]:
pricer.memo[99].shape

KeyError: 99

In [58]:
St = simulator.bs_gpu(r, T, M, N, np.sqrt(theta),S0)
Vt = torch.ones_like(St, )* np.sqrt(theta)

In [59]:
black_scholes(S0, K, T, r, theta, 'put')

0.009991352566430578

In [62]:
pricer.compute_option_prices_counting(St)

tensor(0.0014, device='cuda:0')